In [86]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
from sklearn.svm import SVC
import warnings
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rayir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rayir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rayir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [87]:
dftrans = pd.read_csv('BETTER30.csv')
stop_words = set(stopwords.words('english'))
wnl = WordNetLemmatizer()

def clean_text(text):
    if isinstance(text, float):
        return ""  
    text = str(text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = text.lower()
    tokens = text.split(" ")
    cleaned = []
    for token in tokens:
        lemmatized_token = wnl.lemmatize(token)
        if lemmatized_token not in stop_words:
            cleaned.append(lemmatized_token)
    return " ".join(cleaned)

scam_labels = [
    'scam', 'suspicious', 'highly_suspicious', 'slightly_suspicious', 'potential_scam',
    'scam_response', 'citing urgency', 'suggesting a dangerous situation', 'dismissive official protocols'
]
non_scam_labels = [
    'neutral', 'legitimate', 'standard_opening', 'identification_request', 'polite_ending',
    'adhering to protocols', 'emphasizing security and compliance', 'ready for further engagement'
]

def categorize_label(label):
    if label in scam_labels:
        return 0
    elif label in non_scam_labels:
        return 1
    else:
        return None 

dftrans['CONTEXT'] = dftrans['CONTEXT'].fillna('unknown')
dftrans['FEATURES'] = dftrans['FEATURES'].fillna('unknown')

dftrans.dropna(subset=['TEXT', 'LABEL'], inplace=True)

if 'ANNOTATIONS' in dftrans.columns:
    dftrans.drop(columns=['ANNOTATIONS'], inplace=True)

dftrans['TEXT'] = dftrans['TEXT'].apply(clean_text)
dftrans['CONTEXT'] = dftrans['CONTEXT'].apply(clean_text)

dftrans['LABEL'] = dftrans['LABEL'].apply(categorize_label)

dftrans.dropna(subset=['LABEL'], inplace=True)

dftrans['COMBINED'] = dftrans['TEXT'] + " " + dftrans['CONTEXT']

X = dftrans['COMBINED']
y = dftrans['LABEL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

model = LinearRegression()
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

y_pred_binary = [1 if pred >= 0.5 else 0 for pred in y_pred]
y_pred_binary_int = [int(pred) for pred in y_pred_binary]

mse = mean_squared_error(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred_binary_int)

print(f"Mean Squared Error: {mse}")
print(f"Accuracy: {accuracy}")
print(f"Predictions: {y_pred_binary[:5]}")
print(f"Actual values: {y_pred_binary_int[:5]}")


Mean Squared Error: 16.671719296084092
Accuracy: 0.6764705882352942
Predictions: [0, 1, 1, 1, 1]
Actual values: [0, 1, 1, 1, 1]
